<a href="https://colab.research.google.com/github/AndresMontesDeOca/Laboratorio3/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Experiments

## Libraries

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.callbacks import Callback


import warnings
# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Ajustar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)

# Si también quieres mostrar más columnas
# pd.set_option('display.max_columns', None)


# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)


## Carga Datos

In [3]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# # Ventas
id = "158aOjqxaNO8l97yA6VWJkek_15YVLMhs"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sell-in.txt')
data_ventas = pd.read_csv("sell-in.txt", sep="\t")
data_ventas['periodo'] = pd.to_datetime(data_ventas['periodo'], format='%Y%m')
data = data_ventas.copy()

# # Productos
id = "15JS_k86LS0sgJXma7BOVXWlyNcMwxdhE"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_productos.txt')
data_productos = pd.read_csv("tb_productos.txt", sep="\t")

# # Stocks
id = "15EV-8f_U7onpA1AcTxxXeD-z8yVR4fQu"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_stocks.txt')
data_stocks = pd.read_csv("tb_stocks.txt", sep="\t")
data_stocks['periodo'] = pd.to_datetime(data_stocks['periodo'], format='%Y%m')

# # Productos a predecir
id = "15LjADctFVwjzQFJvfJGFTEdgZx9xCoId"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('productos_a_predecir.txt')
data_productos_a_predecir = pd.read_csv("productos_a_predecir.txt", sep="\t")



## Filter Data

In [34]:
def filter_data(data_all, data_filter):
    # Filtrar el DataFrame 'data_all' para que solo contenga los 'product_id' presentes en 'data_filter'
    data_filtered = data_all[data_all['product_id'].isin(data_filter['product_id'])]

    return data_filtered

## Group Data

In [33]:
def group_data(data, column):
  grouped_data = data.groupby([column, 'periodo']).sum().reset_index()

  # Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
  pivot_data = grouped_data.pivot(index='periodo', columns=column, values='tn')

  # Asegúrate de que los nombres de las columnas sean strings
  pivot_data.columns = pivot_data.columns.astype(str)

  # Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
  pivot_data.columns.name = None

  return pivot_data

## Fill Nulls

In [40]:
# Jugar con esto, no se si esta bien
def fill_nulls(data):
  # Primero usamos bfill para completar las ordenes mas viejas con los valores de las ordenes mas recientes
  data = data.bfill()
  # Luego completamos con ceros los productos que dejamos de vender, o se discontinuaron
  data = data.fillna(0)

  return data

In [5]:
# # Filtrar el DataFrame 'data' para que solo contenga los 'product_id' presentes en 'data_productos_a_predecir'
# data_filtered = data[data['product_id'].isin(data_productos_a_predecir['product_id'])]


# # Agrupa los datos por 'product_id' y 'periodo', y calcula la suma de 'tn'
# grouped_data = data_filtered.groupby(['product_id', 'periodo']).sum().reset_index()

# # Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
# pivot_data = grouped_data.pivot(index='periodo', columns='product_id', values='tn')

# # Rellena los NaN
# pivot_data = pivot_data.fillna(0)

# # Asegúrate de que los nombres de las columnas sean strings
# pivot_data.columns = pivot_data.columns.astype(str)

# # Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
# pivot_data.columns.name = None

# data_2019 = pivot_data.loc['2019']

## Normalize Data

In [6]:
##########################################################################################
def normalize_data(df, normalization="MinMax"):
    """
    Normaliza cada serie de tiempo (columna) de manera individual usando MinMax o Zscore.

    Args:
        df (pd.DataFrame): DataFrame con series de tiempo de distintos productos, cada columna es un producto.
        normalization (str): Tipo de normalización a aplicar. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        normalized_df (pd.DataFrame): DataFrame con las series normalizadas.
        normalization_params (dict): Diccionario con los parámetros necesarios para desnormalizar cada columna.
            - Para "MinMax": valores min y max de cada columna.
            - Para "Zscore": valores mean y std de cada columna.
    """
    normalization_params = {}
    normalized_df = pd.DataFrame()

    for column in df.columns:
        if normalization == "MinMax":
            min_value = df[column].min()
            max_value = df[column].max()
            normalization_params[column] = {"min": min_value, "max": max_value}
            normalized_df[column] = (df[column] - min_value) / (max_value - min_value)

        elif normalization == "ZScore":
            mean_value = df[column].mean()
            std_value = df[column].std()
            normalization_params[column] = {"mean": mean_value, "std": std_value}
            normalized_df[column] = (df[column] - mean_value) / std_value

        else:
            raise ValueError("Invalid normalization method. Choose 'MinMax' or 'ZScore'.")

    return normalized_df, normalization_params
##########################################################################################
def denormalize_series(normalized_series, normalization_params, normalization="MinMax"):
    """
    Desnormaliza una serie de tiempo usando los valores almacenados.

    Args:
        normalized_series (pd.Series): Serie con los datos normalizados.
        normalization_params (dict): Diccionario con los parámetros necesarios para desnormalizar cada serie.
            - Para "MinMax": valores min y max de cada serie.
            - Para "Zscore": valores mean y std de cada serie.
        normalization (str): Tipo de normalización a deshacer. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        denormalized_series (pd.Series): Serie con los datos desnormalizados.
    """
    denormalized_series = pd.Series(index=normalized_series.index)

    for index in normalized_series.index:
        if str(index) in normalization_params:
            params = normalization_params[str(index)]
        else:
            raise KeyError(f"Index {index} not found in normalization parameters.")

        if normalization == "MinMax":
            min_value = params["min"]
            max_value = params["max"]
            denormalized_series[index] = normalized_series[index] * (max_value - min_value) + min_value

        elif normalization == "ZScore":
            mean_value = params["mean"]
            std_value = params["std"]
            denormalized_series[index] = normalized_series[index] * std_value + mean_value

        else:
            raise ValueError("Invalid normalization method. Choose 'MinMax' or 'ZScore'.")

    return denormalized_series
##########################################################################################

## Split Data

In [7]:
def split_data_all(data):
  data_train = data.loc['2017-01':'2018-12']
  data_valid = data.loc['2019-01':'2019-12']
  return data_train, data_valid

## Window Data

In [8]:
def window_dataset(sequence, data_split, window_size, batch_size, n_future, shuffle_buffer=1000, seed=None):
    """Generates dataset windows for multi-step forecasting in a multivariable context.

    Args:
      sequence (array-like): Contains the values of the time series, where each element is an array of feature values.
      data_split (str): Specifies if the dataset is for training or validation/test.
      window_size (int): The number of time steps to include in the feature.
      batch_size (int): The batch size.
      n_future (int): The number of future steps to predict.
      shuffle_buffer (int): Buffer size to use for the shuffle method.
      seed (int, optional): Random seed for reproducibility.

    Returns:
      tf.data.Dataset: TF Dataset containing time windows.
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(sequence)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + n_future, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + n_future))

    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:window_size], window[window_size:]))

    if data_split == 'train':
        # Shuffle the training data to improve generalization
        dataset = dataset.shuffle(shuffle_buffer, seed=seed)
    else:
        # Cache the validation/test data for improved performance
        dataset = dataset.cache()

    # Create batches of windows and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


## Callbacks

In [9]:
#############################################################################

class MAEThresholdCallback(Callback):
    def __init__(self, threshold=0.15):
        super(MAEThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_mae = logs.get('val_mae')
        if val_mae is not None and val_mae <= self.threshold:
            print(f'\nEpoch {epoch+1}: Validation MAE has reached {val_mae:.4f}, stopping training.')
            self.model.stop_training = True

def MyCallbacks(model_name, patience):
    earlystop = tf.keras.callbacks.EarlyStopping('val_loss', patience=patience, restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'ckpts/{model_name}-' + '{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss')
    mae_threshold_callback = MAEThresholdCallback(threshold=0.015)
    return [earlystop] #, checkpoint] #, mae_threshold_callback]

#############################################################################

## Model Design

In [10]:
#############################################################################
def compile_model(new_model, loss):
  new_model.compile(optimizer='adam', loss=loss, metrics=['mae']) # metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
  print(new_model.summary())
  return new_model
#############################################################################
def MyModel(loss, window_size, n_future, n_features):
    new_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((window_size, n_features)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=False)),
        # tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(n_features * n_future, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
        ])
    return compile_model(new_model, loss)

# Pipeline

## Data Preprocessing

In [12]:
data['customer_id'].value_counts()

customer_id
10001    24914
10004    24234
10003    23927
10007    23257
10002    23224
         ...  
10591        5
10559        5
10632        2
10633        2
10637        2
Name: count, Length: 597, dtype: int64

In [17]:
data_by_product

,20001,20002,20003,20004,20005,20006,20007,20008,20009,20010,...,21248,21252,21256,21259,21262,21263,21265,21266,21267,21276
periodo,,,,,,,,,,,,,,,,,,,,,
2017-01-01,934.77,550.16,1063.46,555.92,494.27,528.41,464.67,523.05,378.08,441.26,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-02-01,798.02,505.89,752.12,508.20,551.43,599.19,638.63,631.71,475.24,562.98,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-03-01,1303.36,834.74,917.17,489.91,563.90,868.34,840.83,937.42,456.07,718.45,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-04-01,1069.96,522.35,525.83,512.05,662.59,565.32,741.17,626.52,338.72,607.73,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-05-01,1502.20,843.44,620.48,543.37,515.59,813.18,858.04,849.49,455.37,760.00,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-06-01,1520.07,968.16,744.09,590.51,528.59,698.32,850.76,739.24,548.52,1032.41,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-07-01,1030.67,845.39,785.12,569.88,625.85,670.78,767.24,563.83,406.52,573.89,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-08-01,1267.39,619.71,864.22,1042.53,1068.02,781.41,874.87,837.32,411.07,651.39,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12
2017-09-01,1316.95,1065.35,1465.83,1259.65,1247.89,599.27,912.11,671.25,464.71,940.83,...,0.46,0.38,0.54,0.53,0.47,0.53,0.02,0.02,0.22,0.12


In [16]:
# data
# data_productos
# data_stocks
# data_productos_a_predecir
window_size = 6
n_future = 2
batch_size = 4
normalization = 'MinMax'

# #########################################################################
# Old Pipeline
# #########################################################################
# data_all = group_data(data, data_productos_a_predecir)
# data_all_norm, data_all_norm_params = normalize_data(data_all, normalization=normalization)
# data_all_norm['20001'].describe()
# data_train, data_valid = split_data_all(data_all_norm)
# print(data_train.shape)
# print(data_valid.shape)
# data_train = data_all_norm
# data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)



## Model Train

In [ ]:
# # # # #########################################################################
# # # # # Neural Netowrk Model
# # # #########################################################################
# model_name = 'TimeSeries'
# loss = 'mse'
# patience = 30
# epochs = 500
# n_features = data_all.shape[1]

# callbacks = MyCallbacks(model_name, patience)
# model = MyModel(loss, window_size, n_future, n_features)

# history = model.fit(
#     data_train_windowed,
#     # validation_data = data_valid_windowed,
#     # validation_split=0.2,
#     # callbacks = callbacks,
#     epochs=epochs)

# # plot_history(history, 4)
# # save_model(model, model_name, history, data_test_wrangled)
# # show_predictions(model, data_test_wrangled, data_test[n_past:])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 6, 64)             208128    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 32)                10368     
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1560)              51480     
                                                                 
 reshape (Reshape)           (None, 2, 780)            0         
                                                                 
Total params: 269976 (1.03 MB)
Trainable params: 269976 

In [18]:
# # Supongamos que `data_all` contiene tus datos históricos, incluyendo 2019.
# # data_all debe tener la forma (n_samples, n_features)

# # Convertir el DataFrame a un array de NumPy
# data_all_array = data_all.values

# # Extraer la última ventana de datos de 2019 para predecir enero de 2020
# column_names = data_all.columns  # Obtener los nombres de las columnas

# # Extraer los últimos `window_size` meses de 2019
# input_data = data_all_array[-window_size:].reshape((1, window_size, n_features))

# # Predecir enero de 2020
# pred_january = model.predict(input_data)

# # Asegurarse de que la predicción tenga la forma correcta
# pred_january = pred_january.reshape((1, n_future, n_features))

# # Crear un DataFrame para la predicción de enero de 2020
# pred_january_df = pd.DataFrame(pred_january[0], columns=column_names)
# pred_january_df.index = pd.date_range(start='2020-01-01', periods=n_future, freq='MS')

# # Actualizar la ventana de entrada para predecir febrero de 2020
# input_data = np.append(input_data[:, 1:, :], pred_january[:, 0, :].reshape(1, 1, n_features), axis=1)

# # Predecir febrero de 2020
# pred_february = model.predict(input_data)

# # Asegurarse de que la predicción tenga la forma correcta
# pred_february = pred_february.reshape((1, n_future, n_features))

# # Crear un DataFrame para la predicción de febrero de 2020
# pred_february_df = pd.DataFrame(pred_february[0], columns=column_names)
# pred_february_df.index = pd.date_range(start='2020-02-01', periods=n_future, freq='MS')

# # Predije dos veces, una volviendo a entrenar con los datos predichos de Enero 2020, y la otra no.
# pred_1 = pred_january_df.loc['2020-02-01']

# pred_1_denorm = denormalize_series(pred_1, data_all_norm_params, normalization=normalization)
# data_pred1_denorm = pred_1_denorm.reset_index()
# data_pred1_denorm.columns = ['product_id', 'tn']
# data_pred1_denorm.to_csv('pred_1.csv', index=False)